<a href="https://colab.research.google.com/github/sikalovaliza/ml/blob/main/LogReg_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Логистическая регрессия

Обучим логистическую регрессию для предсказания того, откликнется клиент  
на рекламное предложение (target = 1) или нет (target = 0).

## Подлючение библиотек, загрузка и обзор данных

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/clients_data.csv')

In [4]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


In [5]:
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)

## Практика

**Задание**  
Выведите на экран количество объектов каждого класса. Сколько процентов объектов относятся к положительному классу?  
Ответ округлите до целого числа (например, если доля объектов положительного класса равна 0.412, в ответ запишите 41,  
имея в виду 41 процент).

In [6]:
# ваш код здесь
unique, counts = np.unique(ytrain, return_counts=True)
class_counts = dict(zip(unique, counts))

positive_class_percentage = (class_counts.get(1, 0) / sum(counts)) * 100

print("Количество объектов каждого класса:", class_counts)
print(f"Процент положительного класса: {positive_class_percentage:.0f}%")


Количество объектов каждого класса: {0: 9403, 1: 1253}
Процент положительного класса: 12%


Обучим логистическую регрессию с параметрами по умолчанию.

In [7]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

In [8]:
lr.fit(Xtrain, ytrain)

prediction = lr.predict(Xtest)

Метрику accuracy не стоит использовать при сильном дисбалансе классов. Поэтому посчитайте f1_score для оценки качества модели на тестовых данных.

f1_score принимает значения от 0 до 1. Чем ближе к 1, тем лучше модель.

In [9]:
from sklearn.metrics import f1_score

# ваш код здесь
f1=f1_score(ytest, prediction)
print(f1)

0.0


**Вопрос**  
Чему равен `f1_score`?

Удивительно, да?

Давайте разберемся, почему качество такое низкое.

Предскажем вероятности классов с помощью обученной логистической регрессии на тестовых данных.

In [10]:
probs_test = lr.predict_proba(Xtest)

probs_test[:10]

array([[0.9386009 , 0.0613991 ],
       [0.90059273, 0.09940727],
       [0.71082843, 0.28917157],
       [0.78458556, 0.21541444],
       [0.87089461, 0.12910539],
       [0.95850917, 0.04149083],
       [0.93632116, 0.06367884],
       [0.96561665, 0.03438335],
       [0.9488919 , 0.0511081 ],
       [0.93090985, 0.06909015]])

По вероятностям видно, что вероятности отнесения к положительному классу очень низкие.

Попробуем изменить порог для перевода вероятности в классы.

**Вопрос**  
Чему равен `f1_score`, если все объекты с вероятностью не меньшей 0.1, относить к положительному классу?

Ответ округлите до сотых.

In [11]:
# ваш код здесь
threshold = 0.1
y_prob=probs_test[:, 1]
y_pred=(y_prob >= threshold).astype(int)
f1=f1_score(ytest, y_pred)
print(f1)

0.25485122897800777


У обученной модели можно посмотреть веса (как и в линейной регрессии).  
Выведем на экран веса модели (`model.coef_`, `model.intercept_`).

In [20]:
lr.coef_, lr.intercept_

(array([[-4.61783944e-02, -1.47053630e-01, -8.61148282e-02,
         -1.54078110e-01,  1.91466235e-01, -1.41682812e-01,
          2.01722727e-05, -6.01033673e-02, -2.31271509e-01,
          2.92923914e-01]]),
 array([-0.24168408]))

Создайте `pd.DataFrame`, где в первом столбце стоят названия признаков, а во втором - их веса (так удобнее анализировать результат).  
Отсортируйте таблицу по убыванию весов.

**Вопрос**  
Какой признак имеет наибольший положительный вес?

In [23]:
# ваш код здесь
weights_df = pd.DataFrame({
    'Feature': Xtest.columns,
    'Weight': lr.coef_[0]
})
weights_df = weights_df.sort_values(by='Weight', ascending=False)
print(weights_df)

             Feature    Weight
9       LOAN_DLQ_NUM  0.292924
4        CHILD_TOTAL  0.191466
6    PERSONAL_INCOME  0.000020
0                AGE -0.046178
7     LOAN_NUM_TOTAL -0.060103
2  SOCSTATUS_PENS_FL -0.086115
5         DEPENDANTS -0.141683
1  SOCSTATUS_WORK_FL -0.147054
3             GENDER -0.154078
8    LOAN_NUM_CLOSED -0.231272


## Бонус

**Задание 1**  
Подберите порог для перевода вероятностей в классы, дающий максимальное значение `f1_score`.

In [26]:
# ваш код здесь
thresholds = np.arange(0, 1, 0.01)
for threshold in thresholds:
  y_prob=probs_test[:, 1]
  y_pred=(y_prob >= threshold).astype(int)
  f1=f1_score(ytest, y_pred)
  print(threshold, f1)

0.0 0.21810378462738977
0.01 0.21823150497755223
0.02 0.21813197571960055
0.03 0.2203187250996016
0.04 0.22615131578947367
0.05 0.23148751357220412
0.06 0.2346368715083799
0.07 0.2373138096440293
0.08 0.24204514549904813
0.09 0.25073920756948553
0.1 0.25485122897800777
0.11 0.26277897768178543
0.12 0.25760568945080997
0.13 0.2591623036649215
0.14 0.25244618395303325
0.15 0.2490272373540856
0.16 0.24417872876022656
0.17 0.23612087139845397
0.18 0.21850079744816586
0.19 0.20977777777777779
0.2 0.1884920634920635
0.21 0.16503800217155265
0.22 0.12903225806451613
0.23 0.11494252873563218
0.24 0.09641873278236915
0.25 0.07580174927113703
0.26 0.05801526717557252
0.27 0.03803486529318542
0.28 0.025889967637540454
0.29 0.0165016501650165
0.3 0.01669449081803005
0.31 0.016863406408094434
0.32 0.013628620102214651
0.33 0.006896551724137931
0.34 0.0034662045060658577
0.35000000000000003 0.0034782608695652175
0.36 0.003484320557491289
0.37 0.0034904013961605585
0.38 0.0034965034965034965
0.39 0.0

**Задание 2**  
Во вложенном цикле подберите одновременно коэффициент регуляризации `C` у логистической регрессии и порог для перевода вероятностей в классы, дающие максимальное значение `f1_score`.

По-хорошему, чтобы не переобучиться, эти величины надо подбирать не по тесту, а по отдельной выборке.

Поэтому разобъем данные изначально на три части: `Xtrain`, `Xval`, `Xtest`.

*   В цикле при подборе `С` и порога будем обучаться по `Xtrain`, а предсказывать и измерять качество по `Xval`.

*   Качество итоговой модели с найденными `C` и порогом измерьте по `Xtest`.

Так не переобучимся!


In [ ]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

# ваш код для подбора C и порога здесь

In [ ]:
model = LogisticRegression(C = ...)

model.fit(Xtrain, ytrain) # обучаемся на всех тренировочных данных

prediction = model.predict_proba(Xtest)

classes = ... # переведите полученные вероятности в классы по найденному порогу

In [ ]:
# вычислите значение f1_score на тестовых данных